## Science Frame Correction

In [13]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib widget
from astropy.io import fits
import pickle

science_list = np.genfromtxt('./group10_WASP-135_20190803/science/science.list', dtype=str)


### Testing the code on a subset of images
In lab we tested our reduction pipeline and tools only on a small subsample of images beacuse of reduced available space on the computer. Now we will use the full sample.

In [14]:
science_test_list = science_list[:30] #Nel caso volessimo ridurre basta selezionare solo alcune immagini di science list

### Data Reduction Steps
Science **data reduction** includes the following steps:
1. Multiplication by GAIN (as always)
2. Bias Subtraction
3. Division by flat

Therefore we have the following **error sources**:
1. Readout Noise
2. Bias Error
3. Flat Error
4. Photon Noise

In order to procede we load the necessary quantities.

In [15]:
# Prendiamo i dati salvati
median_bias = pickle.load(open("median_bias.p", "rb"))
median_normalized_flat = pickle.load(open("median_normalized_flat.p", "rb"))
median_normalized_flat_errors = pickle.load(open("median_normalized_flat_errors.p", "rb"))

# Prendiamo i dati rimanenti dai notebook 01 e 02
bias_std = 1.33 # [e] = photoelectrons
readout_noise = 7.10 # [e] = photoelectrons
gain = 1.91 #[e/ADU] from the header file

For the photon noise, we use the method we employed for the flat. The photon noise must be computed **after** removing the bias but **before** correcting for the flat field. In fact the error must be calculated on the actual number of photons received on the detector, not on the photons emitted by the source.

The error on the science_debiased is given by the sum in quadrature of the errors:

##### Debiased Error
$\sigma_{debiased} = \sqrt{\sigma_{rdn}^2 + \sigma_{bias}^2 + \sigma_{photon}^2}$

The photon noise follows a Poissonian distribution so it is given by the square root of the counts $\sqrt{\texttt{science\_debiased}}$:

$\sigma_{debiased} = \sqrt{\sigma_{rdn}^2 + \sigma_{bias}^2 + \texttt{science\_debiased}}$

##### Corrected Error
For science correction, the corrected intensity is given by:

$\text{science\_corrected} = \frac{\text{science\_debiased}}{\text{median\_normalized\_flat}}$

We apply the error propagation rule for division. If a quantity $z$ is given by the ratio of two variables $x$ and $y$,

$\frac{\sigma_z}{z} = \sqrt{\left(\frac{\sigma_x}{x}\right)^2 + \left(\frac{\sigma_y}{y}\right)^2}$

Applying this formula to our variables:

$\frac{\sigma_{\text{science\_corrected}}}{\text{science\_corrected}} =
\sqrt{\left(\frac{\sigma_{\text{science\_debiased}}}{\text{science\_debiased}}\right)^2 +
\left(\frac{\sigma_{\text{median\_normalized\_flat}}}{\text{median\_normalized\_flat}}\right)^2}$

Finally, multiplying both sides by $\text{science\_corrected}$, we obtain the final expression:

$\sigma_{\text{science\_corrected}} =
\text{science\_corrected} \times \sqrt{
\left(\frac{\sigma_{\text{science\_debiased}}}{\text{science\_debiased}}\right)^2 +
\left(\frac{\sigma_{\text{median\_normalized\_flat}}}{\text{median\_normalized\_flat}}\right)^2
}$




In [16]:
for science_name in science_list:
    """Load the science frames, multiply them by the gain, subtract the median_bias,
    divide by the flat. Next we compute the errors."""
    science_fits = fits.open('./group10_WASP-135_20190803/science/' + science_name)
    science_data = science_fits[0].data * gain
    science_fits.close()

    science_debiased = science_data - median_bias # Sottraggo il bias
    science_corrected = science_debiased / median_normalized_flat # Correggo per il flat
    
    science_debiased_errors = np.sqrt(readout_noise**2 + bias_std**2 + science_debiased)
    science_corrected_errors = science_corrected * np.sqrt((science_debiased_errors/science_debiased)**2 + (median_normalized_flat_errors/median_normalized_flat)**2)
    

C:\Users\Lorenzo\AppData\Local\Temp\ipykernel_1632\808184146.py:9: RuntimeWarning: divide by zero encountered in divide
  science_corrected = science_debiased / median_normalized_flat # Correggo per il flat
C:\Users\Lorenzo\AppData\Local\Temp\ipykernel_1632\808184146.py:9: RuntimeWarning: invalid value encountered in divide
  science_corrected = science_debiased / median_normalized_flat # Correggo per il flat
C:\Users\Lorenzo\AppData\Local\Temp\ipykernel_1632\808184146.py:12: RuntimeWarning: divide by zero encountered in divide
  science_corrected_errors = science_corrected * np.sqrt((science_debiased_errors/science_debiased)**2 + (median_normalized_flat_errors/median_normalized_flat)**2)
C:\Users\Lorenzo\AppData\Local\Temp\ipykernel_1632\808184146.py:12: RuntimeWarning: invalid value encountered in multiply
  science_corrected_errors = science_corrected * np.sqrt((science_debiased_errors/science_debiased)**2 + (median_normalized_flat_errors/median_normalized_flat)**2)


### Saving the images

We created a new directory $\texttt{correct}$ where to store the new files. We want to use the same name of the files for our new frames, without the .fits extension.
".fits" is 5 characters, we can take the strin identifying each file name and remove the last five characters and add the new ones which will be $\texttt{\_corr.p}$

In [17]:
# Esempio
# new_name = './correct' + science_name[:-5] + '_corr.p'
# print(science_name)
# print(science_name[:-5])
# new_name = science_name[:-5] + '_corr.p'
# print(new_name)

for science_name in science_test_list:
    new_name = './group10_WASP-135_20190803/correct/' + science_name[:-5] + '_corr.p'
    pickle.dump(science_corrected, open(new_name, 'wb'))
    new_name = './group10_WASP-135_20190803/correct/' + science_name[:-5] + '_corr_error.p'
    pickle.dump(science_corrected_errors, open(new_name, 'wb'))
    